In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='segnet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.51 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/8aa1839925464b57badcf5eb376749b9



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect
from Model import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_no_intersect_1conv_64_channels_30epoch', type=str, help="name of the log") #debug model_intersect
parser.add_argument("-gpu", default='0', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=30, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsect(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 30
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/segnet_no_intersect_1conv_64_channels_30epoch
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images_no_intersect already exists, skipping download
[0.05999042 0.94000958]
Instructions for updating:
Colocations handled automatically by placer.
GGG
(?, 32, 128, 1)
(?, 32, 128, 1)
(?, 32, 128, 64)
111111
(?, 16, 64, 64)
(?, 16, 64, 64)
22222
(?, 8, 32, 64)
(?, 8, 32, 64)
33333
(?, 4, 16, 64)
(?, 4, 16, 64)
44444
(?, 2, 8, 64)
(?, 2, 8, 64)
d4444444
(?, 4, 16, 64)
d333333
(?, 8, 32, 64)
d22222
(?, 16, 64, 64)
d111111
(?, 32, 128, 64)
cv
(?, 32, 128, 2)
(2,)
(?, 32, 128, 2)
loss: ()
INFO:tensorflow:Summary name conv_classifier/weight_loss (raw) is illegal; using conv_classifier/weight_loss__raw_ instead.
INFO:tensorflow:Summary name loss/cross_entropy (r

COMET ERROR: Failed to extract parameters from Estimator.init()


toto_loss_shape: Tensor("loss/total_loss:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Python: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
Tensorflow: 1.12.0-rc0
Ran global_variables_initializer
Epoch: 1  Training...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


TRAIN: Batch: 0.0 Loss: 0.11146697
accuracy = 0.472690
mean IU  = 0.270558
    class # 0 capture rate = 0.443340 
    class # 1 capture rate = 0.868980 
TRAIN: Batch: 0.003908998514580564 Loss: 0.06381499
accuracy = 0.699766
mean IU  = 0.401246
    class # 0 capture rate = 0.695972 
    class # 1 capture rate = 0.771014 
TRAIN: Batch: 0.007817997029161129 Loss: 0.05774217
accuracy = 0.899980
mean IU  = 0.509248
    class # 0 capture rate = 0.940753 
    class # 1 capture rate = 0.235642 
TRAIN: Batch: 0.011726995543741693 Loss: 0.054413356
accuracy = 0.876193
mean IU  = 0.554503
    class # 0 capture rate = 0.890417 
    class # 1 capture rate = 0.650761 
TRAIN: Batch: 0.015635994058322257 Loss: 0.048649214
accuracy = 0.771932
mean IU  = 0.462884
    class # 0 capture rate = 0.766850 
    class # 1 capture rate = 0.864662 
TRAIN: Batch: 0.019544992572902823 Loss: 0.050670788
accuracy = 0.663028
mean IU  = 0.396221
    class # 0 capture rate = 0.646488 
    class # 1 capture rate = 0.89

...

TRAIN: Batch: 0.574622781643343 Loss: 0.008470656
accuracy = 0.963689
mean IU  = 0.773587
    class # 0 capture rate = 0.962420 
    class # 1 capture rate = 0.986842 
TRAIN: Batch: 0.5785317801579235 Loss: 0.0105779655
accuracy = 0.949503
mean IU  = 0.782070
    class # 0 capture rate = 0.945868 
    class # 1 capture rate = 0.989697 
TRAIN: Batch: 0.5824407786725041 Loss: 0.011182731
accuracy = 0.955284
mean IU  = 0.753145
    class # 0 capture rate = 0.953817 
    class # 1 capture rate = 0.979724 
TRAIN: Batch: 0.5863497771870847 Loss: 0.00685459
accuracy = 0.974851
mean IU  = 0.821316
    class # 0 capture rate = 0.974230 
    class # 1 capture rate = 0.986269 
TRAIN: Batch: 0.5902587757016652 Loss: 0.0062990617
accuracy = 0.976316
mean IU  = 0.823205
    class # 0 capture rate = 0.975964 
    class # 1 capture rate = 0.983127 
TRAIN: Batch: 0.5941677742162458 Loss: 0.009427785
accuracy = 0.954756
mean IU  = 0.774468
    class # 0 capture rate = 0.952198 
    class # 1 capture rat

TRAIN: Batch: 0.7661637088577906 Loss: 0.012055038
accuracy = 0.946695
mean IU  = 0.748564
    class # 0 capture rate = 0.944443 
    class # 1 capture rate = 0.977666 
TRAIN: Batch: 0.7700727073723712 Loss: 0.008482444
accuracy = 0.965617
mean IU  = 0.799377
    class # 0 capture rate = 0.964553 
    class # 1 capture rate = 0.981986 
TRAIN: Batch: 0.7739817058869518 Loss: 0.01343117
accuracy = 0.953709
mean IU  = 0.748800
    class # 0 capture rate = 0.953710 
    class # 1 capture rate = 0.953692 
TRAIN: Batch: 0.7778907044015323 Loss: 0.006542148
accuracy = 0.971357
mean IU  = 0.824507
    class # 0 capture rate = 0.970161 
    class # 1 capture rate = 0.989654 
TRAIN: Batch: 0.7817997029161129 Loss: 0.01312697
accuracy = 0.950507
mean IU  = 0.746197
    class # 0 capture rate = 0.949556 
    class # 1 capture rate = 0.965024 
TRAIN: Batch: 0.7857087014306935 Loss: 0.00805294
accuracy = 0.963571
mean IU  = 0.780626
    class # 0 capture rate = 0.962362 
    class # 1 capture rate =

TRAIN: Batch: 0.9577046360722383 Loss: 0.0072064423
accuracy = 0.975828
mean IU  = 0.814619
    class # 0 capture rate = 0.976341 
    class # 1 capture rate = 0.965517 
TRAIN: Batch: 0.9616136345868188 Loss: 0.00970999
accuracy = 0.964230
mean IU  = 0.782311
    class # 0 capture rate = 0.963822 
    class # 1 capture rate = 0.971129 
TRAIN: Batch: 0.9655226331013994 Loss: 0.009043623
accuracy = 0.970604
mean IU  = 0.824680
    class # 0 capture rate = 0.970104 
    class # 1 capture rate = 0.977905 
TRAIN: Batch: 0.96943163161598 Loss: 0.005734056
accuracy = 0.971117
mean IU  = 0.808550
    class # 0 capture rate = 0.969698 
    class # 1 capture rate = 0.996334 
TRAIN: Batch: 0.9733406301305606 Loss: 0.0067076636
accuracy = 0.968455
mean IU  = 0.780890
    class # 0 capture rate = 0.967518 
    class # 1 capture rate = 0.987507 
TRAIN: Batch: 0.9772496286451411 Loss: 0.0073094
accuracy = 0.974633
mean IU  = 0.858631
    class # 0 capture rate = 0.973993 
    class # 1 capture rate =

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 96.485411%. Class 0 capture: 96.414255%. Class 1 capture: 97.611066%
Character error rate not improved
Done with training at epoch 30 sigoptObservation=0.9688151283302641
